# LightGlue-ONNX Evaluation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fabio-sim/LightGlue-ONNX/blob/main/evaluation/lightglue-onnx.ipynb) [![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/fabio-sim/LightGlue-ONNX/main/evaluation/lightglue-onnx.ipynb)

The inference time of [LightGlue-ONNX](https://github.com/fabio-sim/LightGlue-ONNX) is measured and compared to the [original PyTorch implementation](https://github.com/cvg/LightGlue).

## Install Dependencies

Ensure that you are connected to a GPU Runtime. Otherwise, go to `Runtime` -> `Change runtime type` and select the GPU hardware accelerator.

In [ ]:
!nvidia-smi
!nvcc --version
!lsb_release -a

In [ ]:
!pip install -q torch==2.0.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q kornia==0.6.12 onnx==1.14.0 onnxruntime-gpu==1.15.1 numpy opencv-python matplotlib einops
# !pip install -q flash-attn==1.0.8 --no-build-isolation  # Time-consuming (~30 minutes)

In [ ]:
# Install TensorRT
!wget https://developer.nvidia.com/downloads/compute/machine-learning/tensorrt/secure/8.6.1/local_repos/nv-tensorrt-local-repo-ubuntu2204-8.6.1-cuda-11.8_1.0-1_amd64.deb
!sudo dpkg -i nv-tensorrt-local-repo-ubuntu2204-8.6.1-cuda-11.8_1.0-1_amd64.deb
!sudo cp /var/nv-tensorrt-local-repo-ubuntu2204-8.6.1-cuda-11.8/*-keyring.gpg /usr/share/keyrings/
!sudo cp /var/nv-tensorrt-local-repo-ubuntu2204-8.6.1-cuda-11.8/nv-tensorrt-local-0628887B-keyring.gpg /usr/share/keyrings/
!sudo apt-get update
!sudo apt-get install tensorrt python3-libnvinfer-dev
!dpkg-query -W tensorrt

## Code & Data

Clone the repository for the evaluation script.

In [ ]:
!git clone https://github.com/fabio-sim/LightGlue-ONNX

The [MegaDepth test set](https://drive.google.com/file/d/12yKniNWebDHRTCwhBNJmxYMPgqYX3Nhv/view?usp=drive_link) (403 image pairs) provided by [LoFTR](https://github.com/zju3dv/LoFTR/blob/master/docs/TRAINING.md) will be used for measuring the inference times. Download the data and extract.

In [ ]:
!cd LightGlue-ONNX && gdown 12yKniNWebDHRTCwhBNJmxYMPgqYX3Nhv
!cd LightGlue-ONNX && tar -xvf megadepth_test_1500.tar

## ONNX Export

If you haven't exported any ONNX models, please run the export script first. Otherwise, please upload the models to the local `LightGlue-ONNX/weights` folder with the filename formatted as `{extractor_type}_{max_num_keypoints}.onnx`.

In [ ]:
!cd LightGlue-ONNX && python export.py \
  --extractor_type superpoint \
  --extractor_path weights/superpoint_512.onnx \
  --dynamic --max_num_keypoints 512

In [ ]:
# For TensorRT
!cd LightGlue-ONNX && python tools/symbolic_shape_infer.py \
  --input weights/superpoint_lightglue.onnx \
  --output weights/superpoint_lightglue.onnx \
  --auto_merge

## Evaluation

Measure the average inference time of the original PyTorch implementation with the default configuration for LightGlue.

In [ ]:
!cd LightGlue-ONNX && python eval.py torch \
  --img_size 1024 \
  --extractor_type superpoint \
  --max_num_keypoints 512 \
  --device cuda

In [ ]:
!cd LightGlue-ONNX && python eval.py ort \
  --img_size 1024 \
  --extractor_type superpoint \
  --max_num_keypoints 512 \
  --device cuda

# References
1. Philipp Lindenberger, Paul-Edouard Sarlin, Marc Pollefeys: “LightGlue: Local Feature Matching at Light Speed”, 2023; [arXiv:2306.13643](https://arxiv.org/abs/2306.13643).
2. Zhengqi Li, Noah Snavely: “MegaDepth: Learning Single-View Depth Prediction from Internet Photos”, 2018; [arXiv:1804.00607](https://arxiv.org/abs/1804.00607).
3. Jiaming Sun, Zehong Shen, Yuang Wang, Hujun Bao, Xiaowei Zhou: “LoFTR: Detector-Free Local Feature Matching with Transformers”, 2021; [arXiv:2104.00680](https://arxiv.org/abs/2104.00680).